# Transfer Learning

**1. What is Transfer Learning ?**

*Ans :* `Is is the method of using the already trained model on the dataset similar to the trained model data.`

In [9]:
# TensorFlow and tf.keras
import tensorflow as tf

# Helper libraries
import numpy as np
from tensorflow.keras import initializers
from tensorflow.python.keras import activations

print(tf.__version__)

# downloading fashion_mnist data
fashion_mnist = tf.keras.datasets.fashion_mnist

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

train_images = train_images / 255.0

test_images = test_images / 255.0       

activation = tf.keras.activations.tanh

2.5.0


In [16]:
model = tf.keras.Sequential([
tf.keras.layers.Flatten(input_shape=(28, 28)),
tf.keras.layers.Dense(128, activation=activation),
tf.keras.layers.Dense(10)
])

model.compile(optimizer='adam',loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=['accuracy'])

# model summary
model.summary()



model.fit(train_images, train_labels, epochs=10)

test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)

print('test_loss', test_loss)
print('test_accuracy', test_acc)

model.save('model.h5')

## Predicting image is coat or not

In [11]:
# index of 'coat' in train_labels is 4 

coat_train_lbl = train_labels==4
coat_train_lbl

array([False, False, False, ..., False, False, False])

In [12]:
coat_test_lbl = test_labels == 4
coat_test_lbl

array([False, False, False, ..., False, False, False])

**Without Transfer Learning**

In [14]:
model1 = tf.keras.Sequential([
tf.keras.layers.Flatten(input_shape=(28, 28)),
tf.keras.layers.Dense(128, activation=activation),
tf.keras.layers.Dense(1)
])

loss = tf.keras.losses.binary_crossentropy
model1.compile(loss=loss,
                  optimizer = tf.keras.optimizers.SGD(learning_rate=1e-3),
                  metrics = ['accuracy'])

# model summary
model1.summary()

history = model1.fit(train_images, coat_train_lbl, epochs=10,
                    validation_data=(test_images, coat_test_lbl), verbose=2)

2021/06/15 09:45:50 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'a9f71512e0994667ab1aee7b7a27c3a6', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current tensorflow workflow


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_3 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 128)               100480    
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 129       
Total params: 100,609
Trainable params: 100,609
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
1875/1875 - 6s - loss: 1.5428 - accuracy: 0.8999 - val_loss: 1.5428 - val_accuracy: 0.9000
Epoch 2/10
1875/1875 - 4s - loss: 1.5424 - accuracy: 0.9000 - val_loss: 1.5426 - val_accuracy: 0.9000
Epoch 3/10
1875/1875 - 4s - loss: 1.5424 - accuracy: 0.9000 - val_loss: 1.5426 - val_accuracy: 0.9000
Epoch 4/10
1875/1875 - 5s - loss: 1.5423 - accuracy: 0.9000 - val_loss: 

**With Transfer Learning**

In [17]:
pretrained_fmnist_model = tf.keras.models.load_model('model.h5')
pretrained_fmnist_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               100480    
_________________________________________________________________
dense_3 (Dense)              (None, 10)                1290      
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________


In [18]:
for layer in pretrained_fmnist_model.layers:
  print(f"{layer.name}: {layer.trainable}")

flatten_1: True
dense_2: True
dense_3: True


In [19]:
for layer in pretrained_fmnist_model.layers[:-1]:
  layer.trainable = False

In [20]:
for layer in pretrained_fmnist_model.layers:
  print(f"{layer.name}: {layer.trainable}")

flatten_1: False
dense_2: False
dense_3: True


In [21]:
lower_pretrained_layers = pretrained_fmnist_model.layers[:-1]

new_model = tf.keras.models.Sequential(lower_pretrained_layers)
new_model.add(
    tf.keras.layers.Dense(1, activation='sigmoid')
)

In [22]:
new_model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               100480    
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 129       
Total params: 100,609
Trainable params: 129
Non-trainable params: 100,480
_________________________________________________________________


In [23]:
loss = tf.keras.losses.binary_crossentropy
new_model.compile(loss=loss,
                  optimizer = tf.keras.optimizers.SGD(learning_rate=1e-3),
                  metrics = ['accuracy'])

In [24]:
new_model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               100480    
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 129       
Total params: 100,609
Trainable params: 129
Non-trainable params: 100,480
_________________________________________________________________


In [25]:
history = new_model.fit(train_images, coat_train_lbl, epochs=10,
                    validation_data=(test_images, coat_test_lbl), verbose=2)

2021/06/15 09:48:41 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'f8b19c4b1b524262a36d04ae8f0e09c8', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current tensorflow workflow


Epoch 1/10
1875/1875 - 5s - loss: 0.2786 - accuracy: 0.9071 - val_loss: 0.2083 - val_accuracy: 0.9210
Epoch 2/10
1875/1875 - 3s - loss: 0.1857 - accuracy: 0.9312 - val_loss: 0.1742 - val_accuracy: 0.9321
Epoch 3/10
1875/1875 - 3s - loss: 0.1605 - accuracy: 0.9377 - val_loss: 0.1588 - val_accuracy: 0.9348
Epoch 4/10
1875/1875 - 3s - loss: 0.1469 - accuracy: 0.9424 - val_loss: 0.1495 - val_accuracy: 0.9395
Epoch 5/10
1875/1875 - 3s - loss: 0.1381 - accuracy: 0.9461 - val_loss: 0.1432 - val_accuracy: 0.9429
Epoch 6/10
1875/1875 - 3s - loss: 0.1317 - accuracy: 0.9488 - val_loss: 0.1386 - val_accuracy: 0.9437
Epoch 7/10
1875/1875 - 3s - loss: 0.1269 - accuracy: 0.9505 - val_loss: 0.1350 - val_accuracy: 0.9456
Epoch 8/10
1875/1875 - 3s - loss: 0.1231 - accuracy: 0.9521 - val_loss: 0.1323 - val_accuracy: 0.9464
Epoch 9/10
1875/1875 - 3s - loss: 0.1201 - accuracy: 0.9531 - val_loss: 0.1299 - val_accuracy: 0.9466
Epoch 10/10
1875/1875 - 3s - loss: 0.1175 - accuracy: 0.9541 - val_loss: 0.1280 - 

## Observations : 
- 